In [ ]:
#| default_exp widget


In [ ]:
#| export
from __future__ import annotations


## Widgets

`ipywidgets` utils.

# Prologue

In [ ]:
#| export
import asyncio
import time
from functools import wraps

import ipywidgets as W


In [ ]:
#| export

from olio.basic import _get_globals


In [ ]:
import threading

import traitlets as T
from fastcore.test import *
from IPython.display import clear_output
from IPython.display import display
from IPython.display import Markdown


----


# cleanupwidgets
> Helper to properly cleanup ipywidget instances by closing their comms.

When working with ipywidgets in notebooks, each widget creates a comm channel with the kernel. During heavy development, it's better to close the widgets, to avoid memory leaks and kernel issues.


In [ ]:
#| export

def cleanupwidgets(*ws, mod: str|None=None, clear=True):
    from IPython.display import clear_output
    glb = _get_globals(mod or __name__)
    for w in ws:
        _w = glb.get(w) if isinstance(w, str) else w
        if _w:
            try: _w.close()  # type: ignore
            except: pass
    if clear: clear_output(wait=False)

In [ ]:
_b = W.Button()
test_eq(_b.comm is not None, True)
cleanupwidgets('_b')
test_is(_b.comm, None)

In [ ]:
_b = W.Button()
test_eq(_b.comm is not None, True)
cleanupwidgets('_b')
test_is(_b.comm, None)

----
# Asynchronous Widgets

Two scenarios where we'd like widget-related code to run without blocking the kernel from acting on other execution requests.

1. Pausing code to wait for user interaction with a widget in the frontend
2. Updating a widget in the background

In [ ]:
cleanupwidgets('slider')

display(slider := W.IntSlider(max=10))

def work(slider):
    start = time.time()
    print(f"waiting for slider to reach {slider.max}...", end='')
    while True:
        print('.', end='')
        time.sleep(0.5)
        if (time.time() - start) > 5: print('timeout'); break

work(slider)

IntSlider(value=0, max=10)

waiting for slider to reach 10.............timeout


Try to change the slider. You can, the front-end is responsive, but the kernel is blocked from running other code, including handling messages from the front-end.

In [ ]:
cleanupwidgets('progress')

display(progress := W.FloatProgress(value=0.0, min=0.0, max=1.0))

async def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.05)
        progress.value = float(i+1)/total

await work(progress)

FloatProgress(value=0.0, max=1.0)

In [ ]:
print(progress.value)

1.0


Async doesn't help, the kernel is still blocked.

### Waiting for user interaction
> Pausing code to wait for user interaction with a widget in the frontend

#### Event loop integration

If we take advantage of the event loop integration IPython offers, we can have a nice solution async/await syntax.

We define a new function that returns a future for when a widget attribute changes.

In [ ]:
#| export

def wait_for_change(widget:W.Widget, value:str):
    future = asyncio.Future()
    def getvalue(change):
        future.set_result(change.new)
        widget.unobserve(getvalue, value)
    widget.observe(getvalue, value)
    return future

And we finally get to our function where we will wait for widget changes. We'll do 10 units of work, and pause after each one until we observe a change in the widget. Notice that the widget's value is available to us, since it is what the `wait_for_change` future has as a result.

Run this function, and change the slider 10 times.

In [ ]:
cleanupwidgets('slider1', 'out1')

slider1 = W.IntSlider()
out1 = W.Output()

async def f():
    for i in range(10):
        out1.append_stdout('did work ' + str(i))
        x = await wait_for_change(slider1, 'value')
        out1.append_stdout(' - async function continued with value ' + str(x) + '\n')
asyncio.ensure_future(f())
print('aaa')
display(slider1, out1)

aaa


IntSlider(value=0)

Output()

Note that this is not blocking the kernel from running other code. We can run other cells, or even other widgets.

In [ ]:
cleanupwidgets('slider2', 'out2')

slider2 = W.IntSlider()
out2 = W.Output()

def test():
    async def f():
        for i in range(10):
            out2.append_stdout('did work ' + str(i))
            x = await wait_for_change(slider2, 'value')
            out2.append_stdout(' - async function continued with value ' + str(x) + '\n')
    asyncio.ensure_future(f())

test()
display(slider2, out2)

IntSlider(value=0)

Output()

#### Generator approach
> Updating a widget in the background

If you can't take advantage of the async/await syntax, or you don't want to modify the event loop, you can also do this with generator functions.

First, we define a decorator which hooks a generator function up to widget change events.

In [ ]:
#| export

def yield_for_change(widget, attribute):
    """Pause a generator to wait for a widget change event.
        
    This is a decorator for a generator function which pauses the generator on yield
    until the given widget attribute changes. The new value of the attribute is
    sent to the generator and is the value of the yield.
    """
    def f(iterator):
        @wraps(iterator)
        def inner():
            i = iterator()
            def next_i(change):
                try: i.send(change.new)
                except StopIteration as e: widget.unobserve(next_i, attribute)
            widget.observe(next_i, attribute)
            # start the generator
            next(i)
        return inner
    return f

Then we set up our generator.

In [ ]:
cleanupwidgets('slider3')
display(slider3 := W.IntSlider())

@yield_for_change(slider3, 'value')
def f():
    for i in range(10):
        print('did work %s'%i, end=' ')
        x = yield
        print('- generator function continued with value %s'%x)
f();

IntSlider(value=0)

did work 0 

- generator function continued with value 2
did work 1 - generator function continued with value 3
did work 2 - generator function continued with value 8
did work 3 - generator function continued with value 9
did work 4 - generator function continued with value 11
did work 5 - generator function continued with value 17
did work 6 - generator function continued with value 21
did work 7 - generator function continued with value 23
did work 8 - generator function continued with value 25
did work 9 - generator function continued with value 26


#### Modifications

The above two approaches both waited on widget change events, but can be modified to wait for other things, such as button event messages (as in a "Continue" button), etc.

In [ ]:
cleanupwidgets('btn')

class Clickable(W.Button):
    clicked = T.Int(0)
    def __init__(self, *args, on_click=None, **kwargs):
        super().__init__(*args, **kwargs)
        if on_click is None:
            self.on_click(lambda b: b.set_trait('clicked', b.clicked + 1))
            T.dlink((self, 'clicked'), (self, 'description'), lambda x: f'{x}')
        else:
            self.on_click(on_click)

display(btn := Clickable(description='0'))

@yield_for_change(btn, 'clicked')
def f():
    for i in range(10):
        print('did work %s'%i, end=' '  )
        x = yield
        print('- generator function continued with value %s'%x)
f();

Clickable(description='0', style=ButtonStyle())

did work 0 

- generator function continued with value 1
did work 1 - generator function continued with value 2
did work 2 - generator function continued with value 3
did work 3 - generator function continued with value 4
did work 4 - generator function continued with value 5
did work 5 - generator function continued with value 6
did work 6 - generator function continued with value 7
did work 7 - generator function continued with value 8
did work 8 - generator function continued with value 9
did work 9 - generator function continued with value 10


In [ ]:
cleanupwidgets('btn2', 'out3')
out3 = W.Output()

display(btn2 := Clickable(description='0'), out3)

def f():
    async def f():
        for i in range(10):
            out3.append_stdout('did work ' + str(i))
            x = await wait_for_change(btn2, 'clicked')
            out3.append_stdout(' - async function continued with value ' + str(x) + '\n')
    asyncio.ensure_future(f())

f()

Clickable(description='0', style=ButtonStyle())

Output()

In [ ]:
cleanupwidgets('txt', 'out4')
out4 = W.Output()

display(txt := W.Text(continuous_update=False), out4)

def f():
    async def f():
        while True:
            x = await wait_for_change(txt, 'value')
            out4.append_stdout(' - async function continued with value ' + str(x) + '\n')
            if x == 'exit': break
            txt.value = ''
    asyncio.ensure_future(f())

f()

Text(value='', continuous_update=False)

Output()

### Updating a widget in the background

Sometimes you'd like to update a widget in the background, allowing the kernel to also process other execute requests. We can do this with threads. In the example below, the progress bar will update in the background and will allow the main kernel to do other computations.

In [ ]:
cleanupwidgets('progress')

progress = W.FloatProgress(value=0.0, min=0.0, max=1.0)

def work(progress):
    total = 100
    for i in range(total):
        time.sleep(0.01)
        progress.value = float(i+1)/total

thread = threading.Thread(target=work, args=(progress,))
display(progress)
thread.start()

FloatProgress(value=0.0, max=1.0)

In [ ]:
print(17)

17


----

Now we want to handle the inverse problem, where we want to block the kernel from running other code while waiting for a widget to change.

Consider Python `input` function, which blocks until the user enters some input.

In [ ]:
name = input("Enter your name:")
print(f"It's very important to know your name, {name}!")

It's very important to know your name, asd!


If you're running this on VSCode, you'll notice that `input` works as expected, but its UX is unwieldy with a nearly invisible popout far above the cell.

No problem, `ipywidgets` has a `Text` widget that can be used to get user input.

In [ ]:
cleanupwidgets('text')

print('(Stop cell execution when you tire of waiting)')

text = W.Text(placeholder='Input your name; enter to submit', continuous_update=False)
display(text)

while True:
    if text.value: break
    time.sleep(0.5)
print(f"It's very important to know your name, {name}!")

(Stop cell execution when you tire of waiting)


Text(value='', continuous_update=False, placeholder='Input your name; enter to submit')

KeyboardInterrupt: 

The widget is responsive, but as the kernel is blocked, the messages coming from the front-end are not processed.

No problem, Asyncio to the rescue.

In [ ]:
cleanupwidgets('text')

async def wait_for_text(text):
    cnt = 0
    while cnt < 10:
        if text.value: return text.value
        await asyncio.sleep(0.5)
        cnt += 1
    return 'unknown'

text = W.Text(placeholder='Enter your name; enter to submit', continuous_update=False)
display(text)

await wait_for_text(text)
print(f"It's very important to know your name, {name}!")

Text(value='', continuous_update=False, placeholder='Enter your name; enter to submit')

CancelledError: 

You can try lots of convoluted solutions, but by design is very difficult to block the kernel using widgets. Crafting modal UIs with `ipywidgets` is not simple.

**TL;DR**: Use [jupyter-ui-poll](https://github.com/Kirill888/jupyter-ui-poll/tree/develop)  
> Block Jupyter cell execution while interacting with widgets.

We want to solve the following problem:

1. Display User Interface in **Jupyter** using ``ipywidgets`` or similar
2. Wait for data to be entered (this step is surprisingly non-trivial to implement)
3. Use entered data in cells below

You want to implement a notebook like the one below

```python

   # cell 1
   ui = make_ui()
   display(ui)
   data = ui.wait_for_data()

   # cell 2
   do_things_with(data)

   # cell 3.
   do_more_tings()
```

And you want to be able to execute `Cells -> Run All` menu option and still get correct output.

Jupyter assists in implementing your custom `ui.wait_for_data()` poll loop.
If you have tried implementing such workflow in the past you'll know that it is not that simple. If you haven't, see [Technical Details](https://github.com/Kirill888/jupyter-ui-poll/tree/develop?tab=readme-ov-file#technical-details) for an explanation on why it's hard and how `jupyter-ui-poll` solves it.

Quick, self contained example:

In [ ]:
import time
from ipywidgets import Button
from jupyter_ui_poll import ui_events

# Set up simple GUI, button with on_click callback
# that sets ui_done=True and changes button text
ui_done = False

def on_click(btn):
    global ui_done
    ui_done = True
    btn.description = '👍'

btn = Button(description='Click Me')
btn.on_click(on_click)
display(btn)

# Wait for user to press the button
with ui_events() as poll:
    while ui_done is False:
        poll(10)  # React to UI events (up to 10 at a time)
        print('.', end='')
        time.sleep(0.1)
print('done')

Button(description='Click Me', style=ButtonStyle())

................done


Now we can develop our pretty `input` function:

In [ ]:
async def get_user_input(prompt='', placeholder='Write; enter to submit', timeout=10.):
    style = {'description_width':'auto', 'width':'80%'}
    w = W.Text(placeholder=placeholder, continuous_update=False, style=style)
    if prompt: w.description = f"{prompt}:"
    display(w)
    answer = v = w.value
    start_time = time.time()
    async with ui_events() as ui_poll:
        while answer == v:
            if (time.time() - start_time) > timeout: break
            await ui_poll(10)
            await asyncio.sleep(0.2)  # Simulate async processing
            answer = w.value
    clear_output()
    return answer.lower()

In [ ]:
answer = await get_user_input(placeholder='Input your name; enter to submit')
Markdown(f"Your intervention has saved the Universe and beyond, **{name}**!")

Your intervention has saved the Universe and beyond, **aaa**!

# Colophon
----


In [ ]:
import fastcore.all as FC
import nbdev
from nbdev.clean import nbdev_clean


In [ ]:
if FC.IN_NOTEBOOK:
    nb_path = '20_widgets.ipynb'
    nbdev_clean(nb_path)
    nbdev.nbdev_export(nb_path)
